In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# 사전 학습된 모델 : VGG16

In [ ]:
from google.colab import files

# 이미지 업로드
uploaded = files.upload()  # 업로드 창에서 이미지 선택
img_path = list(uploaded.keys())[0]
img = image.load_img(img_path, target_size=(224, 224))

# 시각화
plt.imshow(img)
plt.axis('off')
plt.title("Input Image")
plt.show()

# 이미지 전처리 (VGG16 입력 형태 맞추기)
x = image.img_to_array(img)      # (224,224,3)
x = np.expand_dims(x, axis=0)    # (1,224,224,3)
x = preprocess_input(x)          # 픽셀값을 VGG16 학습 시 입력 스케일로 변환

# 사전 학습된 VGG16 모델 불러오기 (ImageNet 가중치 사용)
# include_top=True → 1000개 ImageNet 클래스 분류기 포함
model = VGG16(weights='imagenet', include_top=True)

# 예측
preds = model.predict(x)

# 예측 결과 디코딩 (Top-5 클래스)
labels = decode_predictions(preds, top=5)[0]

print("Top-5 Predicted Classes:")
for rank, (imagenet_id, label, prob) in enumerate(labels, start=1):
    print(f"{rank}. {label:15s} ({prob*100:.2f}%)")

In [ ]:
model.summary()

## 특징 추출

In [ ]:
from tensorflow.keras import models, layers

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
base_model.trainable = False    # 특징 추출부 고정

model = models.Sequential([
   base_model,
   layers.Flatten(),
   layers.Dense(256, activation='relu'),
   layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary()

## Fine Tuning (상위 층 일부 재학습)

In [ ]:
from tensorflow.keras import models, layers

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
base_model.trainable = True

# 마지막 Conv 2층만 학습, 나머지는 학습 안함
for layer in base_model.layers[:-2] :
    layer.trainable = False

model = models.Sequential([
   base_model,
   layers.Flatten(),
   layers.Dense(256, activation='relu'),
   layers.Dense(3, activation='softmax')
])